# 合成回归数据
:label:`sec_synthetic-regression-data`

机器学习全部都是从数据中提取信息。因此你可能会疑惑，我们能从合成数据中学到什么呢？虽然我们可能本质上并不关心我们自己在人工数据生成模型中嵌入的模式，但这些数据集对于教学目的仍然有用，可以帮助我们评估学习算法的属性，并确认我们的实现按预期工作。例如，如果我们创建的数据其正确参数是预先已知的，那么我们可以检查我们的模型是否确实能够恢复这些参数。

In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

## 生成数据集

为了简洁起见，本示例将在低维度下工作。
以下代码片段生成1000个样本，
每个样本的2维特征从标准正态分布中抽取。
生成的设计矩阵$\mathbf{X}$属于$\mathbb{R}^{1000 \times 2}$。
我们通过应用*真实*线性函数生成每个标签，
并通过独立且同分布于每个样本的加性噪声$\boldsymbol{\epsilon}$来破坏它们：

(**$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \boldsymbol{\epsilon}.$$**)

为方便起见，我们假设$\boldsymbol{\epsilon}$是从均值$\mu= 0$、
标准差$\sigma = 0.01$的正态分布中抽取的。
请注意，为了面向对象设计，
我们将代码添加到`d2l.DataModule`子类的`__init__`方法中（在:numref:`oo-design-data`中介绍）。
允许设置任何额外的超参数是一种良好实践。
我们使用`save_hyperparameters()`来实现这一点。
`batch_size`将在稍后确定。

In [2]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

在下面，我们将真实参数设置为 $\mathbf{w} = [2, -3.4]^\top$ 和 $b = 4.2$。
之后，我们可以将我们估计的参数与这些*真实值*进行对比。

In [3]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

[**`features`中的每一行都是$\mathbb{R}^2$中的一个向量，而`labels`中的每一行都是一个标量。**] 让我们看看第一个条目。

In [4]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([-0.9066,  1.0911]) 
label: tensor([-1.3190])


## 读取数据集

训练机器学习模型通常需要多次遍历数据集，
每次抓取一小批量的样本。
这些数据随后被用来更新模型。
为了说明这个过程是如何工作的，我们
[**实现 `get_dataloader` 方法，**]
通过 `add_to_class` 将其注册到 `SyntheticRegressionData` 类中（在 :numref:`oo-design-utilities` 中介绍）。
它（**接受一个批量大小、一个特征矩阵和一个标签向量，并生成大小为 `batch_size` 的小批量。**）
因此，每个小批量由特征和标签组成的元组构成。
需要注意的是，我们必须留意自己是在训练模式还是验证模式：
在前者中，我们希望以随机顺序读取数据，
而在后者中，能够以预定义的顺序读取数据
对于调试目的可能很重要。

In [5]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

为了建立一些直观认识，让我们检查第一个小批次的数据。每个特征的小批次都提供了其大小和输入特征的维度。同样，我们的标签小批次将有一个由`batch_size`决定的匹配形状。

In [6]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


虽然看似无害，调用 `iter(data.train_dataloader())` 展示了 Python 面向对象设计的强大之处。请注意，我们在创建 `data` 对象 *之后* 向 `SyntheticRegressionData` 类中添加了一个方法。尽管如此，该对象仍然能够从类的功能的 *事后* 增加中受益。

在迭代过程中，我们获得不同的小批量数据，直到整个数据集被耗尽（你可以试试）。虽然上面实现的迭代对于教学目的来说很好，但在某些方面效率低下，这可能会在处理实际问题时给我们带来麻烦。例如，它要求我们将所有数据加载到内存中，并且需要进行大量的随机内存访问。深度学习框架中内置的迭代器要高效得多，它们可以处理诸如存储在文件中的数据、通过流接收的数据以及即时生成或处理的数据等来源。接下来让我们尝试使用内置迭代器来实现相同的方法。

## 简洁实现数据加载器

与其自己编写迭代器，不如[**调用框架中现有的 API 来加载数据。**] 与之前一样，我们需要一个包含特征 `X` 和标签 `y` 的数据集。除此之外，我们在内置的数据加载器中设置 `batch_size` 并让它高效地处理样本的洗牌工作。

In [7]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

In [8]:
@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

新的数据加载器的行为与之前的完全一样，只是它更高效并且增加了一些功能。

In [9]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


例如，框架API提供的数据加载器支持内置的`__len__`方法，因此我们可以查询其长度，即批次的数量。

In [10]:
len(data.train_dataloader())

32

## 摘要

数据加载器是一种方便的方法，可以将加载和处理数据的过程抽象出来。这样，同一个机器学习*algorithm*就可以处理许多不同类型和来源的数据，而无需进行修改。数据加载器的一个优点是它们可以组合使用。例如，我们可能正在加载图像，然后有一个后处理过滤器对它们进行裁剪或其他方式的修改。因此，数据加载器可以用来描述整个数据处理流程。

对于模型本身，二维线性模型是我们可能遇到的最简单的模型之一。它使我们能够在不必担心数据量不足或方程组欠定的情况下测试回归模型的准确性。我们将在下一节中充分利用这一点。


## 练习

1. 如果样本数量不能被批量大小整除会发生什么？如何通过使用框架的API指定不同的参数来改变这种行为？
1. 假设我们要生成一个巨大的数据集，其中参数向量`w`的大小和样本数量`num_examples`都很大。
    1. 如果我们无法在内存中保存所有数据会怎样？
    1. 如果数据存储在磁盘上，你将如何打乱这些数据？你的任务是设计一种*高效*的算法，不需要太多的随机读写。提示：[伪随机置换生成器](https://en.wikipedia.org/wiki/Pseudorandom_permutation)允许你在不显式存储置换表的情况下重新排序 :cite:`Naor.Reingold.1999`。
1. 实现一个数据生成器，每次调用迭代器时都会生成新的数据。
1. 你将如何设计一个每次调用时都生成*相同*数据的随机数据生成器？

[讨论](https://discuss.d2l.ai/t/6663)